In [2]:
import pandas as pd
import pickle
import warnings

%run models.ipynb
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")

In [22]:
dataset = "tacred"

features_with_labels = pd.read_csv(f"/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/features_{dataset}.csv",
                                   index_col=0)
## Sampling 20% of data
features_with_labels_sampled = features_with_labels.sample(frac=0.2, random_state=100)
columns = ['sent_index', 'sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_type_1', 'entity_type_2', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'entity_tag_1',
       'entity_tag_2', 'shortest_distance', 'label']
features_with_labels = features_with_labels[columns]
display(features_with_labels.groupby(by=["label"]).count()["sent_index"])

## Sampling 
# features_with_labels = features_with_labels.sample(frac = 0.3, weights = "label", random_state=101)
# features_with_labels = features_with_labels.sample(frac = 0.3,  
                                                   # weights = features_with_labels.groupby('label')['label'].transform('count'))

X = features_with_labels.iloc[:,:-1]
y =  features_with_labels.iloc[:,-1]

error_rows = X[X.isna().any(axis=1)]
display(error_rows)
X = X.fillna("")

x_train_original, x_test_original, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)

onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)


label
1    16337
Name: sent_index, dtype: int64

,sent_index,sentence_length,root_word,root_index,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,entity_tag_1,entity_tag_2,shortest_distance


In [20]:
pred = logistic_regression(x_train, x_test, y_train, y_test, "binary")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")


--------Logistic Regression-----------


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [13]:
pred = random_forest(x_train, x_test, y_train, y_test,"binary")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [14]:
pred = svm("linear", x_train, x_test, y_train, y_test, "binary")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [15]:
pred = svm("rbf", x_train, x_test, y_train, y_test, "binary")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [16]:
pred = svm("poly", x_train, x_test, y_train, y_test, "binary")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


In [17]:
pred = xgboost(x_train, x_test, y_train, y_test, "binary")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")


In [18]:
pred = adaboost(x_train, x_test, y_train, y_test, "binary")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")
